In [1]:
from strands import Agent
from strands.models.ollama import OllamaModel
from strands_tools import calculator
from pydantic import BaseModel, Field

class MathResult(BaseModel):
    operation: str = Field(description="the performed operation")
    result: int = Field(description="the result of the operation")

model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.2",
)

tool_agent = Agent(
    model=model, 
    name="pirate",
    tools=[calculator]
)

result = tool_agent("What is 42 + 8", structured_output_model=MathResult)


Tool #1: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 42 + 8              │                                                                            │
│  │ Result    │ 50                  │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The answer to the question "What is 42 + 8?" is 50.

/home/locutus/anaconda3/envs/strands-agents-local-test/lib/python3.13/site-packages/strands/event_loop/streaming.py:458: UserWarning: A ToolChoice was provided to this provider but is not supported and will be ignored
  async for event in process_stream(chunks, start_time):



Tool #2: MathResult


In [2]:
print(result.structured_output.result) 

50


In [3]:
print(result.structured_output.operation) 

+


## We can improve the output by making the descriptions more explicit

In [4]:
class MathResult(BaseModel):
    operation: str = Field(
        description="the mathematical expression in format 'number operator number' (e.g., '42 + 8', '10 * 5')"
    )
    result: int = Field(description="the calculated numeric answer")
    operator: str = Field(description="the mathematical operator used: +, -, *, or /")

In [5]:

result = tool_agent("What is 42 + 8", structured_output_model=MathResult)


Tool #3: calculator
 to the question "What is 42 + 8?" is 50.

:ere are the step-by-step calculations

1. Add 42 and 8
2. Result = 50
Tool #4: MathResult


In [6]:
print(result.structured_output.result) 

50


In [7]:
print(result.structured_output.operation) 

42 + 8


In [8]:
print(result.structured_output.operator) 

+


In [9]:
result

AgentResult(stop_reason='tool_use', message={'role': 'assistant', 'content': [{'toolUse': {'toolUseId': 'MathResult', 'name': 'MathResult', 'input': {'operation': '42 + 8', 'operator': '+', 'result': '50'}}}]}, metrics=EventLoopMetrics(cycle_count=6, tool_metrics={'calculator': ToolMetrics(tool={'toolUseId': 'calculator', 'name': 'calculator', 'input': {'expression': '42 + 8', 'mode': 'evaluate', 'precision': '0', 'scientific': 'False', 'variables': '{}', 'wrt': ''}}, call_count=2, success_count=1, error_count=1, total_time=0.005393266677856445), 'MathResult': ToolMetrics(tool={'toolUseId': 'MathResult', 'name': 'MathResult', 'input': {'operation': '42 + 8', 'operator': '+', 'result': '50'}}, call_count=2, success_count=2, error_count=0, total_time=0.0003466606140136719)}, cycle_durations=[0.3257784843444824, 0.382800817489624, 0.5403323173522949, 0.4707648754119873], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='3efbf0a7-7c48-4966-9a3e-9503d0c259

# A note on the ToolChoice error above. 

Ollama supports tool calling but NOT the tool_choice parameter. This means:
* Tools can be called, but you can't force the model to use them
* The model always decides on its own whether to call tools
* Strands tries to set tool_choice for structured output reliability, but Ollama ignores it
